In [72]:
using CUDA, Random, Test
CUDA.allowscalar(false)

In [2]:
include("perm_test.jl")
include("partition.jl")

partition (generic function with 1 method)

In [3]:
Random.seed!(123)
N = 1024
n1, n2 = 12, 8
x1, x2 = CuArray(rand(N, n1)), CuArray(rand(N, n2))

([0.906299638797481 0.18750281732422014 … 0.5941010362656459 0.17323597986665784; 0.44349373245960455 0.498439397828113 … 0.35202055060070614 0.31073836950193956; … ; 0.3589185993287528 0.1114694679461018 … 0.6439746082053515 0.5539659994059574; 0.035927613151065296 0.7081160306383704 … 0.8242544528646908 0.9229874487702134], [0.9191811597933967 0.2925787758936972 … 0.741307376507605 0.9790050449921868; 0.42601923740838954 0.14932329779230624 … 0.7726209816563904 0.4140158502518958; … ; 0.0769627114182928 0.7625196951584902 … 0.16054705848932704 0.926818712518206; 0.48657054103959807 0.015825668052687814 … 0.8230914230335877 0.8461944332763968])

In [4]:
p1, p2 = partition(n1, n2)
p1, p2 = CuArray(p1), CuArray(p2)
p1

125970×12 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1   2   3   4   5   6   7   8   9  10  11  12
 1   2   3   4   5   6   7   8   9  10  11  13
 1   2   3   4   5   6   7   8   9  10  11  14
 1   2   3   4   5   6   7   8   9  10  11  15
 1   2   3   4   5   6   7   8   9  10  11  16
 1   2   3   4   5   6   7   8   9  10  11  17
 1   2   3   4   5   6   7   8   9  10  11  18
 1   2   3   4   5   6   7   8   9  10  11  19
 1   2   3   4   5   6   7   8   9  10  11  20
 1   2   3   4   5   6   7   8   9  10  12  13
 1   2   3   4   5   6   7   8   9  10  12  14
 1   2   3   4   5   6   7   8   9  10  12  15
 1   2   3   4   5   6   7   8   9  10  12  16
 ⋮                   ⋮                   ⋮  
 8   9  10  11  12  13  14  15  16  17  18  20
 8   9  10  11  12  13  14  15  16  17  19  20
 8   9  10  11  12  13  14  15  16  18  19  20
 8   9  10  11  12  13  14  15  17  18  19  20
 8   9  10  11  12  13  14  16  17  18  19  20
 8   9  10  11  12  13  15  16  17  18  19  20
 8   9  10

In [5]:
a, b = x1[1,:], x2[1,:]
wide_lo, wide_hi = tconf(a, b, alpha=0.01, pooled=false)
narrow_lo, narrow_hi = tconf(a, b, alpha=0.1, pooled=false)

(-0.3758134181173361, 0.15103016052040058)

In [6]:
search(a, b, p1, p2, wide_lo, narrow_lo; pooled=false)

-0.5987410951846774

In [7]:
permInterval(a, b, p1, p2, 0)

true

In [69]:
function matrix_row_sum!(out, x, n)
    """out = sum(x, dims=2)"""
    row_idx = (blockIdx().x-1) * blockDim().x + threadIdx().x
    row_sum = 0
    for i = 1:n
        @inbounds row_sum += x[row_idx, i]
    end
    out[row_idx] = row_sum
    return
end

f! (generic function with 2 methods)

In [82]:
y = CuArray{Float64, 1}(zeros(N))
@cuda threads=256 blocks=4 f!(x1, y, 12)
@test isapprox(y, sum(x1, dims=2))

Test Passed
  Expression: isapprox(y, sum(x1, dims = 2))
   Evaluated: isapprox([6.53110238678372, 6.377362795307128, 6.919564267865844, 6.174720415752943, 7.817461484498605, 5.293765801948105, 4.557146998711857, 7.688457256848162, 5.4602616578311665, 4.1431747027884915  …  3.979767404455261, 5.578894708504509, 5.426513198117829, 6.230180953961174, 6.318454449768334, 6.759367514489274, 6.3418690026911335, 7.059379878760508, 7.5059121052625155, 5.577208109503038], [6.53110238678372; 6.3773627953071275; … ; 7.5059121052625155; 5.577208109503038;;])